In [1]:
import requests, json, csv

In [2]:
#Словарь с данными для API
api = {
    'gateway': 'https://manifesto-project.wzb.eu/tools/',
    'key': '?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0'
}

In [23]:
"""
    Обертка для обращений к API
    Аргументы:
        string : api_method – Название метода (https://manifestoproject.wzb.eu/information/documents/api)
        *string : params – Параметры вызова
"""
def api_request(api_method, *params):
    parameters = ''
    if params:
        for i in params:
            parameters += ('&' + i)
    request_url = api['gateway'] + api_method + '.json' + api['key'] + parameters
    print(request_url)
    return json.loads(requests.get(request_url).text)

"""
    Пишет CSV в папку datasheets
    Аргументы:
        list : data – Список данных для записи
        string : name – Имя файла с расширением
"""
def write_csv(data, name):
    path = 'datasheets/' + name
    with open(path, "w", newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in data:
                writer.writerow(line)

"""
    Читает из CSV данные из выбранных колонок
    Аргументы:
        string : name – Имя файла с расширением
        *string : headings – Названия колонок
        (Optional) bool : write_first_row – Записывать ли в список заголовки колонок
"""
def csv_to_list(name, *headings, write_first_row=True):
    with open('datasheets/' + name) as file_obj:
        reader = csv.DictReader(file_obj, delimiter=',')
        result = []
        if write_first_row:
            first_row = []
            for heading in headings:
                first_row.append(heading)
            result.append(first_row)
        for line in reader:
            parsed = []
            for heading in headings:
                parsed.append(line[heading])
            result.append(parsed)
        return result

In [29]:
"""
    Здесь собираем ключи для дальнейшего парсинга метаданных
    Надо будет обернуть в функцию, но мне лень
"""

core_set_from_csv = csv_to_list('core.csv', 'id_perm', 'party', 'date', write_first_row=False)

list_to_parse_metadata = []
for sign in core_set_from_csv:
    item = {
        'id_perm': sign[0],
        'key': sign[1] + '_' + sign[2]
    }
    list_to_parse_metadata.append(item)

In [ ]:
"""
    Здесь получаем метаданные по каждому пункту, проверяем язык документа и его доступность для функции api_texts_and_annotations
    Подходящие документы сохраняем в список
    Когда-нибудь это будет обернуто в красивую функцию с блекджеком и шл***** (красивым выводом и прогрессом выполнения (а может быть даже и виджетом для джупитера))
    А еще у меня сошел с ума яндекс: бесконечно обновляет страницу
    А еще мне надоело ждать пока оно все спарсится
    Мда
    Думаю тот кто будет вызывать этот cell тоже заскучает и сядет писать абсолютно ненужный тут комментарий
    Однако, прошло уже около 4 минут
    Надо бы как-нибудь оптимизировать алгоритм
    И ДА, Я ЗНАЮ ЧТО ПИСАТЬ ТАКИЕ КОММЕНТАРИИ - ЭТО ДУРНОЙ ТОН
    Я специально это оставлю чтобы потом ты, дорогой читатель, понимал как это долго и занудно
    8 минут....
    А это даже веселее чем твиттер
        (!)ИДЕЯ ДЛЯ СТАРТАПА: микроблог на полях рандомного кода с гитхаба
        Тупо бесконечная лента из небольших кусков рандомного кода
        Листаешь такой ленту и среди кода для каких-нибудь атомных реакторов видишь сообщение: "ВАСЯ ПИ**Р"
    20 минут
    Будет забавно, если сейчас что-нибудь сломается, список то не сохранится
    25 минут
    Парсить 4283 записей с средней скоростью 0,5 записи/сек...Ну это п****ц, товарищи
    Может весь код прокоментировать? Как Кругликов на первом курсе просил
    Не, х***я, думай снова
    Даже уже от нечего делать посчитал что весь процесс должен занять почти 36 минут, но судя по ключам в реквестах не прошло еще и половины
    Надо было сделать прогресс-бар: всего каких-то четыре строчки, зато сейчас было бы интереснее
    30 минута уже пошла
    Надо было оптимизировать и передавать в вызове несколько ключей, но я испугался что их, и так еле живой, апи не выдержит таких приколов и умрет
    А так-то тот микроблог на полях кода можно на досуге и написать
"""

list_to_parse_text = []

for sign in list_to_parse_metadata:
    responce = api_request('api_metadata', 'keys[]=' + sign['key'], 'version=2017-2')
    if (responce['items'][0]['language'] == 'english' and 
        responce['items'][0]['annotations'] == 'True'):
            item = {
                'id_perm': sign['id_perm'],
                'key': sign['key'], #на всякий случай, потому что manifesto_id и key могут быть различными (см.документацию API) 
                'manifesto_id': responce['items'][0]['manifesto_id'],
                'md5': responce['items'][0]['md5sum_text']
            }
            list_to_parse_text.append(item)

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11220_194409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11320_194409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11420_194409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11620_194409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11810_194409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11220_194809&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11320_194809&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11420_198209&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11620_198209&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11810_198209&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11220_198509&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11320_198509&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11420_198509&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11520_198509&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11420_201409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11520_201409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11620_201409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11710_201409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=11810_201409&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12220_194510&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12320_194510&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12221_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12320_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12420_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12520_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12620_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12810_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=12951_198109&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13953_194510&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13220_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13320_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13410_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13420_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13620_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=13952_194710&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

...

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53230_201102&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53320_201102&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53520_201102&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53620_201102&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53951_201102&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53110_201602&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=53231_201602&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61320_195611&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61620_195611&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61320_196011&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61620_196011&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61320_196411&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61620_196411&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=61320_196811&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62320_196806&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62420_196806&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62620_196806&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62951_196806&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62320_197210&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62420_197210&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62620_197210&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63320_194609&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63620_194609&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63810_194609&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63320_194912&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63620_194912&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63810_194912&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63320_195104&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63620_198707&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63810_198707&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63320_199003&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63321_199003&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63620_199003&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63810_199003&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=63320_199303&version=2017-2
https://manifesto-project.wzb.eu/tools/api_metadata.json?api_key=5bc5

In [14]:
api_request('api_texts_and_annotations', 'keys[]=41320_2009', 'version=2017-2')

https://manifesto-project.wzb.eu/tools/api_texts_and_annotations.json?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=41320_2009&version=2017-2


{'items': [], 'missing_items': ['41320_2009']}